### The concept of `multiprocessing lock` is similar to the lock in multi threading, without the use of lock when a shared data is accessed by a single process then other process can also access the shared data which will result a corrupt output

https://docs.python.org/3/library/multiprocessing.html#synchronization-between-processes

https://github.com/codebasics/py/blob/master/Multiprocessing/multiprocessing_lock.py

In [1]:
import time
import multiprocessing

### Creating a simple deposit and withdrawal operation without the help of multiprocessing

In [2]:
def deposit_without_mp(balance, amount):
    for i in range(100):
        time.sleep(0.01)
        balance += amount
    return balance

In [3]:
def withdraw_without_mp(balance, amount):
    for i in range(100):
        time.sleep(0.01)
        balance -= amount
    return balance        

In [4]:
balance = 600
balance

600

In [5]:
balance = deposit_without_mp(balance, 5)
balance

1100

In [6]:
balance = withdraw_without_mp(balance, 5)
print('Final balance:', balance)

Final balance: 600


<B> `output`: Although the code is sequential but the output is fine <B>

### This time for the same operation, multiprocessing is used, which is simplifying the operation
The balance is a variable in shared memory now and is not returned by the functions

In [7]:
def deposit_without_lock(balance, amount):
    for i in range(100):
        time.sleep(0.01)
        balance.value += amount

def withdraw_without_lock(balance, amount):
    for i in range(100):
        time.sleep(0.01)
        balance.value -= amount

In [8]:
balance = multiprocessing.Value('i', 600) 

In [9]:
d = multiprocessing.Process(target=deposit_without_lock, args=(balance,5))
w = multiprocessing.Process(target=withdraw_without_lock, args=(balance,5))

d.start()
w.start()

d.join()
w.join()

print('Final balance:', balance.value)

Final balance: 600


<B> `output`: When running for the first time the output may be correct but if we run it again and again then all we get is corrupt output, to solve this problem we use `Lock`  <B>

### Creating the operation with `Lock`
* For process synchronization, we use primitives same as `threading` module
* `acquire()` is to lock the shared data for one process and `release()` is for to release the data after the process done working with the shared data

In [10]:
def deposit_with_lock(balance, amount, lock):
    
    for i in range(100):
        time.sleep(0.01)
        
        lock.acquire() 
        
        balance.value += amount
        
        lock.release()

In [11]:
def withdraw_with_lock(balance, amount, lock):
    
    for i in range(100):
        time.sleep(0.01)
        
        lock.acquire()
        
        balance.value -= amount
        
        lock.release()

In [12]:
balance = multiprocessing.Value('i', 600)
lock = multiprocessing.Lock()

In [13]:
d = multiprocessing.Process(target=deposit_with_lock, 
                            args=(balance, 5, lock))
w = multiprocessing.Process(target=withdraw_with_lock, 
                            args=(balance, 5, lock))

d.start()
w.start()

d.join()
w.join()

print('Final balance:', balance.value)

Final balance: 600


<b> `output`: This time we get the correct time each time we run the operation <b>